In [3]:
import pandas as pd

In [4]:
df = pd.read_parquet('../data/market_data.parquet')

In [5]:
df.head()

,mthcaldt,permno,sharetype,securitytype,securitysubtype,usincflg,issuertype,primaryexch,conditionaltype,tradingstatusflg,...,me,wt,cumretx,ffyear,jdate,szport,bmport,posbm,nonmissport,beme
48,1988-07-29,10001,NS,EQTY,COM,Y,CORP,Q,RW,A,...,6386.00,6200.000000,1.03,1988,1988-07-31,S,H,1.0,1.0,1.207618
49,1988-08-31,10001,NS,EQTY,COM,Y,CORP,Q,RW,A,...,6572.00,6386.000000,1.06,1988,1988-08-31,S,H,1.0,1.0,1.207618
50,1988-09-30,10001,NS,EQTY,COM,Y,CORP,Q,RW,A,...,6362.25,6571.998636,1.02,1988,1988-09-30,S,H,1.0,1.0,1.207618
51,1988-10-31,10001,NS,EQTY,COM,Y,CORP,Q,RW,A,...,6611.75,6323.997695,1.06,1988,1988-10-31,S,H,1.0,1.0,1.207618
52,1988-11-30,10001,NS,EQTY,COM,Y,CORP,Q,RW,A,...,6611.75,6571.999589,1.06,1988,1988-11-30,S,H,1.0,1.0,1.207618


In [6]:
df['permno'].nunique()

20986